# miniporject 3.2

## 카메라 작동 확인

In [10]:
import numpy as np
import cv2
cap = cv2.VideoCapture(0)
cap.set(3,200) # 너비
cap.set(4,150) # 높이
while(True):
    ret, img = cap.read()
    img = cv2.resize(img, None, fx=1.0, fy=1.0, interpolation=cv2.INTER_AREA)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    cv2.imshow('img', img)
    cv2.imshow('gray', gray)
    
    if cv2.waitKey(30) & 0xFF == 27:
        break
cap.release()
cv2.destroyAllWindows()

## 얼굴 감지

In [11]:
import numpy as np
import cv2

faceCascade = cv2.CascadeClassifier('./facetest/FaceDetection/Cascades/haarcascade_frontalface_default.xml')

cap = cv2.VideoCapture(0)
cap.set(3,200) # 너비
cap.set(4,150) # 높이

while True:
    ret, img = cap.read()
    img = cv2.resize(img, None, fx=1.0, fy=1.0, interpolation=cv2.INTER_AREA)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = faceCascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=5, minSize=(20, 20))

    for (x,y,w,h) in faces:
        # 사각형 생성
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,0,255),2)
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = img[y:y+h, x:x+w]  
    cv2.imshow('video',img)

    if cv2.waitKey(30) & 0xFF == 27:
        break
        
cap.release()
cv2.destroyAllWindows()

### 영상저장 및 학습을 위한 관련 폴더 생성

In [13]:
mkdir FacialRecognitionProject

mkdir: FacialRecognitionProject: File exists


In [14]:
mkdir dataset

mkdir: dataset: File exists


## ID 등록

In [4]:
import cv2
import os

cap = cv2.VideoCapture(0)
cap.set(3,200) # 너비
cap.set(4,150) # 높이

face_detector = cv2.CascadeClassifier('./facetest/FaceDetection/Cascades/haarcascade_frontalface_default.xml')

# ID 등록
faceid = input('\n 생성ID를 입력하시오(숫자만 가능)>>>  ')
print("\n 캡쳐중입니다. 카메라를 보고 잠시 기다리시오...")

# 학습을 위해 캡처
count = 0
while(True):
    ret, img = cap.read()
    img = cv2.resize(img, None, fx=1.0, fy=1.0, interpolation=cv2.INTER_AREA)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_detector.detectMultiScale(gray, 1.3, 5)
    for (x,y,w,h) in faces:
        cv2.rectangle(img, (x,y), (x+w,y+h), (255,0,0), 2)     
        count += 1
        
        # 화면을 캡처하여 그레이스케일로 폴더에 저장
        cv2.imwrite("dataset/User." + str(faceid) + '.' + str(count) + ".jpg", gray[y:y+h,x:x+w])
        cv2.imshow('image', img)
    
    if cv2.waitKey(30) & 0xFF == 27:
        break
    # 30장의 사진을 찍으면 종료
    elif count >= 30: 
         break

print("\n 완료")
cap.release()
cv2.destroyAllWindows()

KeyboardInterrupt: Interrupted by user

## 훈련

In [6]:
mkdir trainer
# 터미널로 가서 rm .DS_Store 실행

In [3]:
import cv2
import numpy as np
from PIL import Image
import os

# 저장된 경로 설정
path = 'dataset'
recognizer = cv2.face.LBPHFaceRecognizer_create()
detector = cv2.CascadeClassifier("./facetest/FaceDetection/Cascades/haarcascade_frontalface_default.xml");

# 이미지 학습 관련 함수
def getImagesAndLabels(path):
    imagePaths = [os.path.join(path,f) for f in os.listdir(path)]     
    faceSamples=[]
    ids = []
    for imagePath in imagePaths:
        PIL_img = Image.open(imagePath).convert('L')
        img_numpy = np.array(PIL_img,'uint8')
        id = int(os.path.split(imagePath)[-1].split(".")[1])
        faces = detector.detectMultiScale(img_numpy)
        for (x,y,w,h) in faces:
            faceSamples.append(img_numpy[y:y+h,x:x+w])
            ids.append(id)
    return faceSamples,ids
print ("\n 학습중입니다... 조금만 기다시리시오...")
faces,ids = getImagesAndLabels(path)
recognizer.train(faces, np.array(ids))

# model을 trainer폴더에 trainer.yml로 저장
recognizer.write('trainer/trainer.yml')

# 몇개 종류의 얼굴을 학습했는지 출력
print("\n {0} 개의 얼굴을 학습했습니다. 프로그램을 종료합니다.".format(len(np.unique(ids))))


 학습중입니다... 조금만 기다시리시오...

 3 개의 얼굴을 학습했습니다. 프로그램을 종료합니다.


## 학습된 데이터를 이용하여 얼굴 인식

In [18]:
import cv2
import numpy as np
import os 
recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.read('trainer/trainer.yml')
cascadePath = "./facetest/FaceDetection/Cascades/haarcascade_frontalface_default.xml"
faceCascade = cv2.CascadeClassifier(cascadePath)
font = cv2.FONT_HERSHEY_SIMPLEX

id = 0
# id에 맞게 리스트 구현(id가 1인사람의 이름을 list[1] 자리에 입력)
names = ['None', 'Jinho', 'ChangJun', 'SH'] 

# 비디오 실행
cap = cv2.VideoCapture(0)
cap.set(3,200) # 너비
cap.set(4,150) # 높이
# 얼굴 인식할수있는 최소크기 설정
minW = 0.1*cam.get(3)
minH = 0.1*cam.get(4)

while True:
    ret, img =cap.read()
    img = cv2.resize(img, None, fx=1.0, fy=1.0, interpolation=cv2.INTER_AREA)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    
    faces = faceCascade.detectMultiScale(gray,scaleFactor = 1.2,minNeighbors = 5,minSize = (int(minW), int(minH)))
    
    for(x,y,w,h) in faces:
        cv2.rectangle(img, (x,y), (x+w,y+h), (0,255,0), 2)
        id, confidence = recognizer.predict(gray[y:y+h,x:x+w])
        
        # If confidence is less them 100 ==> "0" : perfect match 
        if (confidence < 100):
            id = names[id]
            confidence = "  {0}%".format(round(100 - confidence))
        else:
            id = "unknown"
            confidence = "  {0}%".format(round(100 - confidence))
        # 관련 이름 입력
        cv2.putText(img, str(id), (x+5,y-5), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)
        # 정확도 입력
        cv2.putText(img, str(confidence), (x+5,y+h-5), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,0), 1)  
    
    cv2.imshow('camera',img) 
    if cv2.waitKey(30) & 0xFF == 27:
        break

print("\n 프로그램을 종료합니다.")
cam.release()
cv2.destroyAllWindows()


 프로그램을 종료합니다.
